# 0.Import
import python and file

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
#%matplotlib inline

In [0]:
header = ["ID","visited_page_url","search_words","use_ip","visited_time","visited_total",
          "language_id","result_numbers","created_person","updated_person","created_at","updated_at"]
df = pd.read_excel (r'/content/drive/My Drive/fs/search.xlsx', names=header)

In [97]:
df.shape

(234184, 12)

#1.EDA


##1.1 check null

In [98]:
df.isnull().sum()

ID                    0
visited_page_url      0
search_words         12
use_ip                0
visited_time          0
visited_total         0
language_id           0
result_numbers        0
created_person        0
updated_person        0
created_at            0
updated_at          387
dtype: int64

In [99]:
df[df['search_words'].isnull()]

,ID,visited_page_url,search_words,use_ip,visited_time,visited_total,language_id,result_numbers,created_person,updated_person,created_at,updated_at
110710,4258628,https://www.fs.com/de-en/?main_page=advanced_search_result&keyword=-SC+UPC+Duplex+Single+Mode+Fibre+Patch&searchSubmit=%EF%85%83,NaN,45.67.109.7,2020-05-12 00:00:00,1,9,28,0,0,2020-05-12 23:34:43,2020-05-12 23:34:45
111902,4259820,https://www.fs.com/de-en/keyword/-SC%20UPC%20Duplex%20Single%20Mode%20Fibre%20Patch&style=image&count=24&page=2&searchSubmit=Search,NaN,45.67.109.7,2020-05-13 00:00:00,1,9,28,0,0,2020-05-13 02:49:20,2020-05-13 02:49:22
141288,4289206,https://www.fs.com/fr/?main_page=advanced_search_result&keyword=%F4%80%80%94%F4%80%80%95%F4%80%80%93%F4%80%80%9B%F4%80%80%98&searchSubmit=%EF%85%83,NaN,185.43.60.130,2020-05-20 00:00:00,1,3,0,0,0,2020-05-20 00:22:45,2020-05-20 00:22:45
153266,4301184,https://www.fs.com/?main_page=advanced_search_result&keyword=%F4%8F%B0%84%F4%8F%B0%A5%F4%8F%B0%99%F4%8F%B0%84%F4%8F%B0%BB%F4%8F%B0%80%F4%8F%B0%82%F4%8F%B0%82%F4%8F%B0%94%F4%8F%B0%82%F4%8F%B1%86%F4...,NaN,73.61.113.226,2020-05-22 00:00:00,1,1,0,184714,184714,2020-05-22 07:15:53,2020-05-22 07:15:53
158043,4305961,https://www.fs.com/au/?main_page=advanced_search_result&keyword=-MLX&searchSubmit=%EF%85%83,NaN,114.134.6.41,2020-05-24 00:00:00,1,9,0,238667,238667,2020-05-24 22:23:10,2020-05-24 22:23:10
164578,4312496,https://www.fs.com/de-en/?main_page=advanced_search_result&keyword=-lx&searchSubmit=%EF%85%83,NaN,213.93.2.84,2020-05-26 00:00:00,1,9,170,108267,108267,2020-05-26 05:45:35,2020-05-26 05:45:36
176247,4324165,https://www.fs.com/?main_page=advanced_search_result&keyword=-M+Clamp&searchSubmit=%EF%85%83,NaN,187.190.185.133,2020-05-28 00:00:00,1,1,2,0,0,2020-05-28 06:54:07,2020-05-28 06:54:08
208693,4356611,https://www.fs.com/de-en/?main_page=advanced_search_result&keyword=-BX-10-U&searchSubmit=%EF%85%83,NaN,80.100.130.167,2020-06-05 00:00:00,1,9,620,0,0,2020-06-05 03:48:18,2020-06-05 03:48:19
212464,4360382,https://www.fs.com/jp/?main_page=advanced_search_result&keyword=-u&searchSubmit=%EF%85%83,NaN,122.216.200.130,2020-06-06 00:00:00,1,8,2683,0,0,2020-06-06 18:44:16,2020-06-06 18:44:17
212465,4360383,https://www.fs.com/jp/?main_page=advanced_search_result&keyword=-u+-d&searchSubmit=%EF%85%83,NaN,122.216.200.130,2020-06-06 00:00:00,2,8,1605,0,0,2020-06-06 18:44:32,2020-06-06 18:44:59


no use to our analysis, will drop

In [0]:
df.dropna(subset=['search_words'], inplace=True)

In [101]:
df.isnull().sum()

ID                    0
visited_page_url      0
search_words          0
use_ip                0
visited_time          0
visited_total         0
language_id           0
result_numbers        0
created_person        0
updated_person        0
created_at            0
updated_at          387
dtype: int64

##1.2 check duplicate

In [102]:
df[df['ID'].duplicated()]

,ID,visited_page_url,search_words,use_ip,visited_time,visited_total,language_id,result_numbers,created_person,updated_person,created_at,updated_at


##1.3 Feature analysis


###1.3.1 Search words

In [103]:
df['search_words'].nunique()

80103

In [104]:
print('Top 10 searched keywords')
tempdf = df['search_words'].value_counts().nlargest(10)#.sum()
print(tempdf)
print('\ntotal = {} out of {}'.format(df['search_words'].value_counts().nlargest(10).sum(),df.shape[0]))

Top 10 searched keywords
11552              1187
sfp                 600
11555               594
11774               498
splitter            448
SFP-10GSR-85        411
66613               404
11773               403
media converter     394
11775               376
Name: search_words, dtype: int64

total = 5315 out of 234172


assuming data is extracted from 14 APR to 12 JUN


In [105]:
vc = df['search_words'].value_counts()
List=vc[vc <= 1].index.tolist()
mapping={}
for s in List:
    mapping[s]='other'
df['search_words2']=df['search_words'].map(lambda x: mapping[x] if x in mapping else x)
df['search_words2'].nunique()

19311

In [0]:
List=vc[vc <= 1].index.tolist()

In [107]:
print('Number of search\n')
vc = df['search_words'].value_counts()
b=0
for i in range(20):          
        a = len(vc[vc == i+1].index.tolist())
        print('{} record search {} times'.format(a, i+1))
        b=b+a
print('total record search less than 20 times = {}'.format(b))

Number of search

60793 record search 1 times
7687 record search 2 times
2923 record search 3 times
1766 record search 4 times
1127 record search 5 times
834 record search 6 times
544 record search 7 times
470 record search 8 times
342 record search 9 times
310 record search 10 times
260 record search 11 times
262 record search 12 times
211 record search 13 times
194 record search 14 times
170 record search 15 times
145 record search 16 times
123 record search 17 times
122 record search 18 times
118 record search 19 times
90 record search 20 times
total record search less than 20 times = 78491


In [108]:
df.head()

,ID,visited_page_url,search_words,use_ip,visited_time,visited_total,language_id,result_numbers,created_person,updated_person,created_at,updated_at,search_words2
0,4147918,https://www.fs.com/?main_page=advanced_search_result&keyword=OM2-LC-ST-SX-FS-XM&searchSubmit=%EF%85%83,OM2-LC-ST-SX-FS-XM,199.117.238.26,2020-04-14 00:00:00,1,1,0,227527,227527,2020-04-14 13:29:18,2020-04-14 13:29:18,other
1,4147919,https://www.fs.com/?main_page=advanced_search_result&keyword=30909&searchSubmit=%EF%85%83,30909,71.175.4.156,2020-04-14 00:00:00,1,1,1,229598,229598,2020-04-14 13:29:27,2020-04-14 13:29:27,30909
2,4147920,https://www.fs.com/?main_page=advanced_search_result&keyword=lc+upc+to+st+simplex&searchSubmit=%EF%85%83,lc upc to st simplex,199.117.238.26,2020-04-14 00:00:00,1,1,24,227527,227527,2020-04-14 13:29:32,2020-04-14 13:29:33,other
3,4147921,https://www.fs.com/mx/?main_page=advanced_search_result&keyword=12369&searchSubmit=%EF%85%83,12369,45.238.180.13,2020-04-14 00:00:00,1,2,1,0,0,2020-04-14 13:29:34,2020-04-14 13:29:34,12369
4,4147922,https://www.fs.com/?main_page=advanced_search_result&keyword=30908+&searchSubmit=%EF%85%83,30908,71.175.4.156,2020-04-14 00:00:00,1,1,1,229598,229598,2020-04-14 13:29:44,2020-04-14 13:29:44,30908


####1.3.1.1 Search words to Product ID

In [109]:
def checkPID(a):
  i = str(a)
  if ((i.isdigit()) and (len(i)==5)):
    return 1
  else:
    return 0
#df['search_words'].astype(str)
df['isProductID'] = df['search_words'].apply(checkPID)
df.sample(3)

,ID,visited_page_url,search_words,use_ip,visited_time,visited_total,language_id,result_numbers,created_person,updated_person,created_at,updated_at,search_words2,isProductID
55451,4203369,https://www.fs.com/?main_page=advanced_search_result&keyword=J9151A&searchSubmit=%EF%85%83,J9151A,59.175.180.242,2020-04-28 00:00:00,1,1,1,129757,129757,2020-04-28 19:06:04,2020-04-28 19:06:04,J9151A,0
137450,4285368,https://www.fs.com/?main_page=advanced_search_result&keyword=mma1l10-cr&searchSubmit=%EF%85%83,mma1l10-cr,24.96.218.54,2020-05-19 00:00:00,1,1,1,0,0,2020-05-19 07:01:02,2020-05-19 07:01:03,other,0
137335,4285253,https://www.fs.com/?main_page=advanced_search_result&keyword=97572&searchSubmit=%EF%85%83,97572,80.151.75.26,2020-05-19 00:00:00,1,1,1,0,0,2020-05-19 06:26:58,2020-05-19 06:26:58,97572,1


In [110]:
print('instance of customer search by actual product ID = {}'.format(df['isProductID'].sum()))

instance of customer search by actual product ID = 80508


In [0]:
df2 = df[df['isProductID'] != 1]

In [112]:
df2.sample(5)

,ID,visited_page_url,search_words,use_ip,visited_time,visited_total,language_id,result_numbers,created_person,updated_person,created_at,updated_at,search_words2,isProductID
39883,4187801,https://www.fs.com/?main_page=advanced_search_result&keyword=S5850-48S6Q&searchSubmit=%EF%85%83,S5850-48S6Q,59.175.180.242,2020-04-24 00:00:00,1,1,1,0,0,2020-04-24 01:36:07,2020-04-24 01:36:07,S5850-48S6Q,0
85880,4233798,https://www.fs.com/?main_page=advanced_search_result&keyword=media+converter&searchSubmit=%EF%85%83,media converter,24.130.59.134,2020-05-06 00:00:00,1,1,24,0,0,2020-05-06 15:32:42,2020-05-06 15:32:42,media converter,0
60506,4208424,https://www.fs.com/de/?main_page=advanced_search_result&keyword=SFP-10G-BX10&searchSubmit=%EF%85%83,SFP-10G-BX10,173.160.217.249,2020-04-29 00:00:00,1,5,24,0,0,2020-04-29 16:56:31,2020-04-29 16:56:32,SFP-10G-BX10,0
114465,4262383,https://www.fs.com/?main_page=advanced_search_result&keyword=media+converter&searchSubmit=%EF%85%83,media converter,87.180.125.96,2020-05-13 00:00:00,1,1,24,0,0,2020-05-13 12:57:20,2020-05-13 12:57:21,media converter,0
87622,4235540,https://www.fs.com/es/?main_page=advanced_search_result&keyword=breakout+10m&searchSubmit=%EF%85%83,breakout 10m,80.102.82.104,2020-05-07 00:00:00,3,2,39,0,0,2020-05-07 00:25:00,2020-05-07 00:26:33,breakout 10m,0


In [113]:
print('Top 10 searched keywords[took out product ID search]')
tempdf = df2['search_words'].value_counts().nlargest(50)#.sum()
print(tempdf)
print('\ntotal = {} out of {}'.format(df2['search_words'].value_counts().nlargest(10).sum(),df2.shape[0]))

Top 10 searched keywords[took out product ID search]
sfp                600
splitter           448
SFP-10GSR-85       411
media converter    394
keystone           375
patch panel        341
attenuator         297
SFP-10G-SR         295
SFP-10GLR-31       293
SFP                262
SFP-10G-LR         261
SFP1G-LX-31        259
SFP-GB-GE-T        248
qsfp               226
SFP1G-SX-85        215
dac                206
GLC-LH-SMD         197
100957             192
coupler            189
gpon               184
pigtail            182
J9150D             176
S3900-24T4S        174
mpo                172
S5850-48S6Q        171
S3900-48T4S        170
fhd                165
rj45               163
dwdm               163
ubiquiti           162
SFP-10G-T          162
S3900              162
pdu                162
mikrotik           160
cwdm               156
loopback           156
S5800-8TF12S       156
N5850-48S6Q        156
cat6               154
breakout           153
breakout cable     146
adap

###1.3.2 Country based on url

In [0]:
pd.options.display.max_colwidth = 200

In [115]:
df['visited_page_url'].tail(20)

234164                                                                     https://www.fs.com/ru/?main_page=advanced_search_result&keyword=72420&searchSubmit=%EF%85%83
234165                                                                    https://www.fs.com/de/?main_page=advanced_search_result&keyword=105883&searchSubmit=%EF%85%83
234166                                                                        https://www.fs.com/?main_page=advanced_search_result&keyword=74481&searchSubmit=%EF%85%83
234167                                                                     https://www.fs.com/ru/?main_page=advanced_search_result&keyword=72448&searchSubmit=%EF%85%83
234168                                                                     https://www.fs.com/ru/?main_page=advanced_search_result&keyword=26389&searchSubmit=%EF%85%83
234169                                                       https://www.fs.com/au/?main_page=advanced_search_result&keyword=FIBER+CLEAVING+TOOL&searchSubmit=%E

filter based on jeremy website. jeremy website belongs to UAT site, should not appear here, hence drop

In [116]:
df[df['visited_page_url'].str.contains("jeremy")].count()

ID                  11
visited_page_url    11
search_words        11
use_ip              11
visited_time        11
visited_total       11
language_id         11
result_numbers      11
created_person      11
updated_person      11
created_at          11
updated_at          11
search_words2       11
isProductID         11
dtype: int64

In [0]:
df.drop(df[df['visited_page_url'].str.contains("jeremy")].index, inplace=True)

In [118]:
df[df['visited_page_url'].str.contains("jeremy")].count()

ID                  0
visited_page_url    0
search_words        0
use_ip              0
visited_time        0
visited_total       0
language_id         0
result_numbers      0
created_person      0
updated_person      0
created_at          0
updated_at          0
search_words2       0
isProductID         0
dtype: int64

In [119]:
df[['homepage','theRest1']] = df['visited_page_url'].str.split(r'com/', n = 1, expand = True) 
df[['theRest1','theRest2']] = df['theRest1'].str.split(r'/?', n = 1, expand = True) 

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1494: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)


In [120]:
df['Country'] = df['theRest1'].map(lambda x: x if (len(x) < 6) and (len(x) > 1) else 'unidentified')
df['Country'].value_counts()

unidentified    149457
de-en            18629
de               18213
au               10874
uk                9454
fr                8211
es                4747
sg                4336
ru                4284
mx                3472
jp                2459
it                  25
Name: Country, dtype: int64

In [0]:
#df['Country_isspace'] = map(lambda x: x.isspace(), df['Country'])

In [145]:
df[df['Country'].str.contains("unidentified")].sample(3)

,ID,visited_page_url,search_words,use_ip,visited_time,visited_total,language_id,result_numbers,created_person,updated_person,created_at,updated_at,search_words2,isProductID,homepage,theRest1,theRest2,Country
69484,4217402,https://www.fs.com/?main_page=advanced_search_result&keyword=40%EF%BC%A7+850nm&searchSubmit=Search,40锛?850nm,223.104.63.32,2020-05-03 00:00:00,1,1,0,0,0,2020-05-03 05:19:13,2020-05-03 05:19:13,40锛?850nm,0,https://www.fs.,?main_page=advanced_search_result&keyword=40%EF%BC%A7+850nm&searchSubmit=Search,None,unidentified
13662,4161580,https://www.fs.com/?main_page=advanced_search_result&keyword=CMH-SFD2U&searchSubmit=%EF%85%83,CMH-SFD2U,27.16.223.184,2020-04-17 00:00:00,1,1,1,0,0,2020-04-17 07:10:48,2020-04-17 07:10:49,CMH-SFD2U,0,https://www.fs.,?main_page=advanced_search_result&keyword=CMH-SFD2U&searchSubmit=%EF%85%83,None,unidentified
204986,4352904,https://www.fs.com/?main_page=advanced_search_result&keyword=SFP-H10GB-CU+&searchSubmit=%EF%85%83,SFP-H10GB-CU,137.69.117.201,2020-06-04 00:00:00,1,1,19,0,0,2020-06-04 11:17:20,2020-06-04 11:17:22,SFP-H10GB-CU,0,https://www.fs.,?main_page=advanced_search_result&keyword=SFP-H10GB-CU+&searchSubmit=%EF%85%83,None,unidentified


###1.3.3 search result based on url

In [0]:
from bs4 import BeautifulSoup, SoupStrainer
import requests

def getData(i):
  url=i

  # Make a GET request to fetch the raw HTML content
  html_content = requests.get(url).text
  #strainer = SoupStrainer('form', attrs={'id': 'form1'})
  # Parse the html content
  soup = BeautifulSoup(html_content, "lxml")
  #mydivs = soup.select('div[class="new_proList_proListTIB"] > span')\
  try:
    mydivs = soup.find('h1', {'class': 'new_proList_proListTit'}).contents[2].strip()
  except:
    #mydivs = soup.find('p', {'class': 'search_404_Sorry'}).text
    mydivs = 0
    #pass
  return mydivs

In [0]:
tempdf = pd.DataFrame({'url':  ['https://www.fs.com/?main_page=advanced_search_result&keyword=40%EF%BC%A7+850nm&searchSubmit=Search', 
                               'https://www.fs.com/?main_page=advanced_search_result&keyword=CMH-SFD2U&searchSubmit=%EF%85%83',
                               'https://www.fs.com/?main_page=advanced_search_result&keyword=SFP-H10GB-CU+&searchSubmit=%EF%85%83'
                               ]})

In [0]:
tempdf['searchResultQTY'] = tempdf['url'].apply(getData)

In [199]:
tempdf[['searchResultQTY','url']]

,searchResultQTY,url
0,0,https://www.fs.com/?main_page=advanced_search_result&keyword=40%EF%BC%A7+850nm&searchSubmit=Search
1,1,https://www.fs.com/?main_page=advanced_search_result&keyword=CMH-SFD2U&searchSubmit=%EF%85%83
2,19,https://www.fs.com/?main_page=advanced_search_result&keyword=SFP-H10GB-CU+&searchSubmit=%EF%85%83


In [0]:
List1 = df['visited_page_url'].apply(getData).tolist()
#df[['searchResultQTY','url']].sample(5)